In [1]:
import numpy as np
import scipy as sp
import scipy.stats as sps
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import gc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dateutil.parser import parse
from datetime import datetime, timedelta
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/Upsell/analysis')
from collections import defaultdict
import statsmodels.stats.api as sms
import time

In [2]:
import yt.wrapper as yt

In [3]:
DATE = lib.get_current_date_as_str()

In [4]:
MAX_LEADS_NUMBER = 50

# CALL Infrormation table

In [5]:
last_call_table = sorted(lib.find_tables_in_hahn_folder("//home/cloud_analytics/dwh/raw/crm/calls"))[-1]
users_table = sorted(lib.find_tables_in_hahn_folder('//home/cloud_analytics/dwh/raw/crm/users'))[-1]

In [6]:
call_info_df = lib.execute_query(f"""
SELECT
    billing_account_id,
    call_time,
    lead_source,
    lead_source_description,
    status,
    user_name
FROM (
SELECT
    parent_id,
    toDateTime(date_start) as call_time,
    status,
    user_name
FROM "{last_call_table}" as calls
LEFT JOIN (
    SELECT
        DISTINCT
            user_name,
            id
    FROM "{users_table}"
) as users
ON calls.assigned_user_id == users.id
WHERE parent_type == 'Leads'
) as calls 
INNER JOIN (
    SELECT
        lead_id,
        argMax(billing_account_id, date_modified) as billing_account_id,
        argMax(lead_source, date_modified) as lead_source,
        argMax(lead_source_description, date_modified) as lead_source_description
    FROM "//home/cloud_analytics/kulaga/leads_cube"
    GROUP BY lead_id
) as crm_lead_info
ON calls.parent_id == crm_lead_info.lead_id
WHERE billing_account_id != ''
FORMAT TabSeparatedWithNames
""")

In [7]:
lib.save_table("crm_call_infromation_for_billings", "//home/cloud_analytics/lunin-dv/crm", call_info_df)

# Business

In [8]:
company_df = lib.execute_query("""
SELECT
    DISTINCT 
    billing_account_id
FROM (
    SELECT
        DISTINCT
        billing_account_id,
        if (ba_person_type like '%company%', 1, is_corporate_card) as is_company
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"

    UNION ALL
    SELECT 
        billing_account_id,
        1 as is_company
    FROM "//home/cloud_analytics/import/crm/business_accounts/data"
    )
WHERE is_company == 1
FORMAT TabSeparatedWithNames
""")

In [9]:
company_bills = set(company_df['billing_account_id'])

In [10]:
len(company_df)

11350

# Additional conditions

In [11]:
condition_df = lib.execute_query(f"""
SELECT
    DISTINCT
    billing_account_id
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
WHERE event == 'ba_created'
AND block_reason NOT IN ('manual', 'mining')
AND segment NOT IN ('Large ISV', 'Medium', 'Enterprise', 'VAR')
AND ba_usage_status != 'service'
AND is_fraud = 0
AND ba_state != 'inactive'
AND ba_state != 'payment_not_confirmed'
FORMAT TabSeparatedWithNames
""")

In [12]:
condition_bills = set(condition_df['billing_account_id'])

In [13]:
len(condition_bills & company_bills)

7114

# Paid + NO ML

In [14]:
paid_no_ml_df = lib.execute_query(f"""
SELECT
    billing_account_id,
    SUM(if(service_name != 'cloud_ai', real_consumption, 0)) as not_ai_consumption
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
WHERE toDate(event_time) >= addDays(toDate('{DATE}'), -14)
and toDate(event_time) < toDate('{DATE}')
GROUP BY billing_account_id
HAVING not_ai_consumption > 0
FORMAT TabSeparatedWithNames
""")

In [15]:
paid_no_ml_bills = set(paid_no_ml_df['billing_account_id'])

In [16]:
len(paid_no_ml_bills)

7715

# CALLS

In [17]:
call_df = lib.execute_query(f"""
SELECT
    billing_account_id,
    if(isNull(last_call_time) or last_call_time = '', ba_created_datetime, last_call_time) as last_call_time,
    if(isNull(for_plateau_users_call_time) or for_plateau_users_call_time = '', 
    ba_created_datetime, last_call_time) as for_plateau_users_last_call_time,
    
    if(isNull(last_source) or last_source = '', 'unknown', last_source) as last_source,
    dateDiff('day', toDate(last_call_time), toDate('{DATE}')) as day_diff,
    dateDiff('day', toDate(for_plateau_users_last_call_time), toDate('{DATE}')) as plateau_day_diff,
    if (day_diff > 70, 1, 0) as contact_more_then_70_days,
    if (plateau_day_diff > 30, 1, 0) as plateau_can_be_used
FROM (
    SELECT
        billing_account_id,
        event_time as ba_created_datetime
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
    WHERE event == 'ba_created'
) as cube
    LEFT JOIN (
    SELECT
        billing_account_id,
        MAX(call_time) as last_call_time,
        argMax(lead_source, call_time) as last_source,
        MAX(if (lead_source in ('cold_calls', 'upsell', 'Marketo'), call_time, null)) 
        as for_plateau_users_call_time
    FROM "//home/cloud_analytics/lunin-dv/crm/crm_call_infromation_for_billings"
    WHERE 
    status == 'Held'
    AND 
    billing_account_id NOT IN 
        (SELECT DISTINCT ba_id FROM "//home/cloud_analytics/export/crm/mql/2019-10-21T10:00:00")
    GROUP BY
        billing_account_id
) as call_info
on cube.billing_account_id == call_info.billing_account_id
FORMAT TabSeparatedWithNames
""")

# Плато

In [18]:
plateau_df = lib.execute_query(f"""
SELECT
    billing_account_id,
    avg(day_consumption) as avg_consumption,
    stddevPop(day_consumption) as std_consumption,
    if(std_consumption / avg_consumption * 100 <= 10.0, 1, 0) as is_plateau
FROM (
    SELECT
        billing_account_id,
        if (isNull(day_consumption), 0, day_consumption) as day_consumption,
        DATE
    FROM (
        SELECT
            billing_account_id,
            arrayJoin(arrayMap(x -> addDays(addDays(toDate('{DATE}'), -14), x),
                         range(14))) as DATE
        FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
        WHERE 
            event == 'ba_created'
        and
            billing_account_id in (
                                    SELECT DISTINCT billing_account_id
                                    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
                                    WHERE event = 'day_use'
                                    AND real_consumption > 0
                                    AND toDate(event_time) >= addDays(toDate('{DATE}'), -14)
                                    AND toDate(event_time) < toDate('{DATE}')
                                  )
    ) as main
    ANY LEFT JOIN (
        SELECT
            billing_account_id,
            toDate(event_time) as DATE,
            SUM(real_consumption) as day_consumption
        FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
        WHERE toDate(event_time) >= addDays(toDate('{DATE}'), -14)
        and toDate(event_time) < toDate('{DATE}')
        and event = 'day_use'
        GROUP BY billing_account_id, DATE
    ) as cons
    ON main.billing_account_id == cons.billing_account_id and main.DATE == cons.DATE
    ORDER BY billing_account_id, DATE
)
GROUP BY billing_account_id
FORMAT TabSeparatedWithNames
""")

# Main information

In [19]:
df = lib.execute_query(f"""
SELECT
    billing_account_id,
    if(phone == '', ifNull(crm_phone, ''), phone) as phone,
    email,
    timezone,
    first_name,
    last_name,
    client_name,
    segment,
    ba_person_type
FROM (
    SELECT
        DISTINCT
        billing_account_id,
        puid,
        segment,
        ba_person_type,
        multiIf(
            user_settings_email LIKE '%@yandex.%' OR user_settings_email LIKE '%@ya.%',
            CONCAT(lower(
                   replaceAll(
                   splitByString('@', assumeNotNull(user_settings_email))[1], 
                   '.', '-'
                   )
                   ), '@yandex.ru'),
            lower(user_settings_email)
        ) as email,
        phone,
        crm_phone,
        multiIf(first_name IS NULL OR first_name = '', 'unknown', first_name) as first_name,
        multiIf(last_name IS NULL OR last_name = '', 'unknown', last_name) as last_name,
        multiIf(account_name IS NULL OR account_name = '', 'unknown', account_name) as client_name,
        ba_person_type
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube" as cube
    ANY LEFT JOIN (
        SELECT
            DISTINCT 
            billing_account_id,
            phone as crm_phone
        FROM "//home/cloud_analytics/import/crm/business_accounts/data"
    ) as crm
    ON cube.billing_account_id == crm.billing_account_id
    WHERE puid != ''
    AND event == 'ba_created'
) as main
ANY LEFT JOIN(
    SELECT
        DISTINCT
        passport_uid as puid,
        timezone
    FROM "//home/cloud_analytics/import/iam/cloud_owners_history"
    WHERE
        puid != ''
) as tm
ON main.puid == tm.puid
FORMAT TabSeparatedWithNames
""")

# BD on vm

In [20]:
bd_on_vm = lib.execute_query("""
SELECT
    billing_account_id,
    arrayStringConcat(data_bases, ',') as data_bases
FROM (
    SELECT
        billing_account_id,
        SUM(multiIf(service_name = 'mdb', real_consumption, 0)) as mdb_paid
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
    WHERE
        event = 'day_use'
        AND real_consumption_vat > 0
    GROUP BY
        billing_account_id
    HAVING mdb_paid <= 0
) as cube
LEFT JOIN (
    SELECT
        billing_account_id,
        arraySort(arrayDistinct(groupArray(db))) as data_bases
    FROM "//home/cloud_analytics/import/network-logs/db-on-vm/data"
    WHERE
        billing_account_id IS NOT NULL
        AND billing_account_id != ''
    GROUP BY
        billing_account_id
) as db
ON cube.billing_account_id == db.billing_account_id
FORMAT TabSeparatedWithNames
""")

# MERGE

In [21]:
main_df = df.copy()
main_df = pd.merge(main_df, call_df, on = 'billing_account_id', how='left')
main_df = pd.merge(main_df, plateau_df, on = 'billing_account_id', how='left')
main_df = pd.merge(main_df, bd_on_vm, on = 'billing_account_id', how='left')
main_df['description'] = main_df['data_bases'].apply(
    lambda x: 'Client Use BD on VM: ' if x == '' or pd.isnull(x) else x)
assert main_df.shape[0] == df.shape[0]
main_df = main_df[main_df['billing_account_id'].isin(company_bills)]
main_df = main_df[main_df['billing_account_id'].isin(condition_bills)]

In [22]:
def lead_source_creator(row):
    if row['is_plateau'] == 1:
        return 'upsell'
    if row['contact_more_then_70_days'] == 1:
        return 'contact more then 70 days'
    return 'other'

In [23]:
main_df['lead_source'] = main_df.apply(lambda row: lead_source_creator(row) , axis=1)

In [24]:
upsell_main_df = main_df[main_df['lead_source'] == 'upsell']
upsell_main_df = upsell_main_df[upsell_main_df['billing_account_id'].isin(paid_no_ml_bills)]

In [25]:
not_contacted_main_df = main_df[main_df['lead_source'] == 'contact more then 70 days']

In [26]:
days_before_to_observe = 70
old_experiment_table_max = sorted(lib.find_tables_in_hahn_folder("//home/cloud_analytics/export/crm/upsale"))[-1]
max_date = old_experiment_table_max.split("/")[-1]
min_date = lib.date_to_string(datetime.now() - timedelta(days=70)) + " 00:00:00"

In [46]:
old_experiment_bills = lib.execute_query(f"""
SELECT DISTINCT billing_account_id
FROM concatYtTablesRange("//home/cloud_analytics/export/crm/upsale")

UNION ALL

SELECT DISTINCT billing_account_id 
FROM "//home/cloud_analytics/smb/upsell_exp/exp_users" WHERE group = 'test'
FORMAT TabSeparatedWithNames
""")

In [47]:
# old_experiment_bills = lib.execute_query(f"""
# SELECT DISTINCT billing_account_id
# FROM concatYtTablesRange("//home/cloud_analytics/export/crm/upsale", '{min_date}', '{max_date}')
# FORMAT TabSeparatedWithNames
# """)

In [ ]:
old_experiment_bills.shape

In [30]:
upsell_final = upsell_main_df[
    ~upsell_main_df['billing_account_id'].isin(old_experiment_bills['billing_account_id'])
]
upsell_final = upsell_final[(upsell_final['phone'] != '') &
                            (upsell_final['plateau_can_be_used'] == 1)]

In [31]:
not_contacted_final = not_contacted_main_df[
    ~not_contacted_main_df['billing_account_id'].isin(old_experiment_bills['billing_account_id'])
]
not_contacted_final = not_contacted_final[not_contacted_final['phone'] != '']

In [32]:
columns = ['billing_account_id', 'email', 'phone', 
           'first_name', 'last_name', 'client_name', 'timezone',
           'lead_source', 'description']

In [33]:
upsell_final = upsell_final[columns]
not_contacted_final = not_contacted_final[columns]
final = lib.concatenate_tables([upsell_final, not_contacted_final])

In [36]:
if final.shape[0] < MAX_LEADS_NUMBER and parse(max_date) < parse(DATE):
    old_experiment_bills_not_full = lib.execute_query(f"""
    SELECT DISTINCT billing_account_id
    FROM concatYtTablesRange("//home/cloud_analytics/export/crm/upsale", '{min_date}', '{max_date}')
    FORMAT TabSeparatedWithNames
    """)
    ########################################################
    upsell_adding_old = upsell_main_df[
        (~upsell_main_df['billing_account_id'].isin(old_experiment_bills_not_full['billing_account_id'])) &
        (~upsell_main_df['billing_account_id'].isin(final['billing_account_id']))
    ]
    upsell_adding_old = upsell_adding_old[(upsell_adding_old['phone'] != '') &
                                          (upsell_adding_old['plateau_can_be_used'] == 1)]
    upsell_adding_old = upsell_adding_old[columns]
    ########################################################
    not_contacted_adding_old = not_contacted_main_df[
        (~not_contacted_main_df['billing_account_id'].isin(old_experiment_bills_not_full['billing_account_id'])) &
        (~not_contacted_main_df['billing_account_id'].isin(final['billing_account_id']))
    ]
    not_contacted_adding_old = not_contacted_adding_old[not_contacted_adding_old['phone'] != '']
    not_contacted_adding_old = not_contacted_adding_old[columns]
    ########################################################
    adding_num = MAX_LEADS_NUMBER - final.shape[0]
    upsell_adding_num = min(adding_num, upsell_adding_old.shape[0])
    adding_num -= upsell_adding_num
    not_contacted_adding_num = min(adding_num, not_contacted_adding_old.shape[0])
    final = lib.concatenate_tables([final, upsell_adding_old.head(upsell_adding_num),
                                   not_contacted_adding_old.head(not_contacted_adding_num)])

In [40]:
final.shape

(0, 9)

In [41]:
table_name = str(datetime.now()).split('.')[0]

In [42]:
if final.shape[0] > 0:
    table_name = str(datetime.now()).split('.')[0]
    lib.save_table(table_name, '//home/cloud_analytics/export/crm/upsale', final)